# Replicaing the Vision Transformer present in the ViT paper
- https://arxiv.org/abs/2010.11929

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision

import tensorboard


In [3]:
torch.__version__, torchvision.__version__

('2.2.2+cu121', '0.17.2+cu121')

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"